In [48]:
import numpy as np
import pandas as pd
from datetime import datetime
from datetime import date

Data Loading

In [39]:

def load_data(file):
    mdro = pd.read_excel(file, sheet_name='MDROs', 
                        dtype={'病歷號碼':'str', '申請編號':'int', '就醫序號':'int'})
    report = pd.read_excel(file, sheet_name='微生物報告', 
                        dtype={'病歷號碼':'str', '申請編號':'int', '就醫序號':'int'})
    info = pd.read_excel(file, sheet_name='就醫資訊', 
                        dtype={'病歷號碼':'str', '就醫序號':'int'})
    anti = pd.read_excel(file, sheet_name='抗生素使用', 
                        dtype={'病歷號碼':'str', '就醫序號':'int'})
    examValue = pd.read_excel(file, sheet_name='檢驗數值', 
                        dtype={'病歷號碼':'str', '就醫序號':'int'})
    
    # data processing 
    mdro = mdro[['病歷號碼', 'time', '就醫序號','申請編號', '菌株', '預測值', '部位', 'sendTime']]
    mdro.rename({'time':'openTime'}, axis=1, inplace=True)
    
    report = report[['病歷號碼', 'bed', '就醫序號','申請編號', 'strain','部位', '抗藥性菌株', 'examTime','reportTime']]
    
    cdss = pd.merge(report, mdro, how='left', on=['病歷號碼', '就醫序號','申請編號', '部位'])
    cdss = cdss.drop_duplicates()
    cdss['minTime'] = cdss.groupby(['病歷號碼', 'bed','就醫序號', '申請編號', '部位', '菌株', '預測值',  'sendTime'])['openTime'].transform('min')
    
    info = info[['病歷號碼', '就醫序號', '性別', '生日', '年齡', '來院日期', '離院日期', '天數', '門急住', '科別代碼', '科別']]
    info.rename({'門急住':'bed'}, axis=1, inplace=True)
    
    cdss_info = pd.merge(cdss, info, how='left', on=['病歷號碼', '就醫序號', 'bed'])
    cdss_info = cdss_info.drop_duplicates()
    
    CDSS = pd.merge(cdss_info, anti, how='left', on=['病歷號碼','就醫序號'])
    CDSS = CDSS.drop_duplicates()
    
    # exam Year & Month
    CDSS["examDate"] = pd.to_datetime( 
                                  pd.to_datetime(CDSS.examTime).dt.year.map(str) + '-' + \
                                  pd.to_datetime(CDSS.examTime).dt.month.map(str) + '-' + \
                                  pd.to_datetime(CDSS.examTime).dt.day.map(str)
                                )
    
    
    return CDSS
    

In [40]:
# 智抗菌平台報表_20220331.xlsx

file = r'C:\Users\User\VSCode\智抗菌平台報表_20220331.xlsx'
CDSS = load_data(file)
# CDSS.head(10)

In [41]:
old_file = r'C:\Users\User\VSCode\智抗菌平台報表_20220309.xlsx'
CDSS_old = load_data(old_file)
CDSS = pd.concat([CDSS_old, CDSS])
CDSS = CDSS.drop_duplicates()

In [42]:
# CDSS.to_excel(r'C:\Users\User\VSCode\CDSS(20220401).xlsx')

In [57]:
SMS = pd.read_excel(r'C:\Users\User\VSCode\簡訊通知時間(0301-0331)給鈞威.xls', dtype={'病歷號_10碼':'str'},
                    sheet_name='0301-0309', )

In [58]:
SMS = SMS[['西元年月', '時間', '病人姓名', '病歷號_10碼', '西元採檢日']]
SMS.rename({'病歷號_10碼':'病歷號', '西元採檢日':'examDate'}, axis=1, inplace=True)


In [59]:
SMS

,西元年月,時間,病人姓名,病歷號,examDate
0,2022-03-01,13:28:00,陳龍鑫,0017662463,2022-02-25
1,2022-03-01,13:28:00,黃裕益,0038131412,2022-02-25
2,2022-03-01,13:28:00,陳俊吉,0030442219,2022-02-25
3,2022-03-01,13:28:00,周嘉豪,0031487807,2022-02-26
4,2022-03-01,15:51:00,周嘉豪,0031487807,2022-02-28
...,...,...,...,...,...
119,2022-03-31,14:35:00,向富義,0037818664,2022-03-30
120,2022-03-31,14:35:00,陳榮梅,0019781710,2022-03-30
121,2022-03-31,14:35:00,林信有,0000944590,2022-03-29
122,2022-03-31,14:35:00,陳黃淑仔,0038128193,2022-03-30
